In [2]:
import csv, datetime, re, json

SUBS = "Monthly Bills: Subscriptions"
GRC = "Everyday Expenses: Groceries"
FUN = "Fun Stuff: UNK"
TRNSP = "Everyday Expenses: Transportation"
HSHLD = "Everyday Expenses: Household Goods"
REST = "Everyday Expenses: Restaurants"
MED = "Everyday Expenses: Medical"
GAS = "Everyday Expenses: Gas"
MOVIES = "Fun Stuff: Movies"
BANKING = "Monthly Bills: Banking"
ELECTRONICS = "Fun Stuff: Gadgets"
MUSIC = "Fun Stuff: Music"
PHONE ="Monthly Bills: Phone"
SPORTS = "Fun Stuff: Sports"
INSURANCE = "Rainy Day Funds: Insurance"
PARKING = "Monthly Bills: Parking"
BOOKS = "Fun Stuff: Books"
CLOTHING = "Everyday Expenses: Clothing"
UTILITIES = "Monthly Bills: Utilities"
COSMETICS = "Everyday Expenses: Cosmetics"
CONSTRUCTION = "Construction"
TAXES = "Monthly Bills: Taxes"
HOTELS = "Fun Stuff: Hotels"
GAMES = "Fun Stuff: Games"

In [3]:
regex1 = "Tranzactie comerciant - Tranz: Nr card [^ ]*, Ref \d*, Suma platita \d*.\d* RON. Comision: 0 RON. Locatie: [^ ]* (?:(?:RO|OM|DE|A|DU) )?(.*) Data_Ora: .*"
regex_foreign = "Tranzactie comerciant - Tranz: Nr card [^ ]*, Ref \d*, Suma platita \d*.\d* (?:EUR|USD), Suma in valuta contului \d*.\d* RON, Suma decontata \d*.\d* (?:EUR|USD) Rata de schimb .* RON. Comision: .* RON. Locatie: (.*). Data_Ora:"
regex_income = "Referinta [^,]*, data valutei \d\d-\d\d-\d\d\d\d, Decontare -Platitor: (.*); .*Beneficiar: .*; RO.*-Detalii: .*"
regex_online = "Tranzactie efectuata prin 24 Banking BCR Referinta [^ ]*, data valutei \d\d-\d\d-\d\d\d\d, Decontare -Platitor: .*-Beneficiar: (.*);"
f = open("./roda.json")

# expense_reader = csv.reader(f)
# expenses = list(expense_reader)
expenses = json.load(f)
#print(expenses)
#NOTICe
# MCH ORADEA might be KFC
processors = [("NIS ORADEA", "Gazprom", GAS),
              ("LIDL", "Lidl", GRC),
              ("PH LOTUS", "Pizza Hut", REST),
              ("CINEMATOGRAF LOTUS", "Cinema Palace", MOVIES),
              ("Comision Exchange", "BCR", BANKING),
              ("Amazon", "Amazon", FUN),
              ("MEDIAGALAXY", "MediaGalaxy", ELECTRONICS),
              ("ALTEX", "Altex", ELECTRONICS),
              ("CARREFOUR", "Carrefour", GRC),
              ("Retragere Numerar", "Transfer: Cash", ""),
              ("NETFLIX.COM", "Netflix", SUBS),
              ("MOL ORADEA", "Mol", GAS),
              ("BANDCAMP", "Bandcamp", MUSIC),
              ("TECHONA", "Meatic", REST),
              ("JUMBO", "Jumbo", HSHLD),
              ("WWW.ORANGE.RO", "Orange", PHONE),
              ("AUCHAN", "Auchan", GRC),
              ("ROUMASPORT", "Decathlon", SPORTS),
              ("eMAG.ro", "eMag", ELECTRONICS),
              ("US FOOD NETWORK", "KFC", REST),
              ("WWW.DECLIC.RO", "DeClic", "Giving: Charitable"),
              ("INST NAT SI DEZVOLTARE BUCURESTI.", "ROTLD", SUBS),
              ("CFR UNATTENDED UNATTENDED", "CFR", TRNSP),
              ("Comision Card", "BCR", BANKING),
              ("LIBRARIILE HUMANITAS", "Humanitas", BOOKS),
              ("Primaria Municipiul Or Oradea", "Primaria Oradea", PARKING),
              ("KAUFLAND", "Kaufland", GRC), 
              ("OMV", "OMV", GAS),
              ("UNIQAONLINE", "UNIQA", INSURANCE),
              ("DEDEMAN", "Dedeman", HSHLD),
              ("Comision Primire", "BCR", BANKING),
              ("Funding", "BCR", BANKING),
              ("ARTIMA", "Carrefour", GRC),
              ("H&M", "H&M", CLOTHING),
              ("SENSIBLU", "Sensiblu", MED),
              ("AVSTORE BUCURESTI", "AVStore", ELECTRONICS),
              ("ANDROMI", "Andromi", GRC),
              ("ORANGE ROMANIA", "Orange", PHONE),
              ("PACT EXIM", "Saladbox",REST),
              ("SCRIPTUM", "Scriptum", BOOKS),
              ("ACTIV FOOD EXPRES", "Seramis Coffee", GRC),
              ("SUBWAY ERA", "Subway", REST),
              ("MEDLENS", "Medlens", MED),
              ("GOOGLE *Google Music g.co/helppay", "Google", SUBS),
              ("KFC ORADEA DT ORADEA", "KFC", REST),
              ("JYSK", "Jysk", HSHLD),
              ("BEEMINDER PORTLAND", "Beeminder", SUBS),
              ("taxify.eu", "Taxify", TRNSP),
              ("CEL.RO", "Cel.Ro", ELECTRONICS),
              ("BIOCLINICA ", "Bioclinica", MED),
              ("MEGAIMAGE ", "MegaImage", GRC),
              ("ROMPETROL ", "Rompetrol", GAS),
              ("HIPMENU ", "HipMenu", REST),
              ("COMPANIA DE APA", "Compania de Apa", UTILITIES),
              ("ASHI BND SRL-D CLUJ-NAPOCA", "Grocery Store next to Office", GRC),
              ("PANEMAR MORARIT", "Panemar", GRC),
              ("PANEMAR MAGAZIN", "Panemar", GRC),
              ("MOBILPAY*TAPPTITUDE", "AmParcat", PARKING),
              ("RER ECOLOGIC SERVICE ", "RER", UTILITIES),
              ("SAMSARA FOOD CLUJ NAPOCA", "Samsara", REST),
              ("DMA QUALITY FOOD BAIA MARE", "Saladbox", REST),
              ("M.D.O. PEYATO SRL", "Magazin Naturist Era", MED),
              ("Orsay-RO ORADEA", "Orsay", CLOTHING),
              ("DM DROGERIE MARKT ", "DM", COSMETICS),
              ("C & A MODA", "C&A", CLOTHING),
              ("DM FIL 08 >ORADEA", "DM", COSMETICS),
              ("NOODLE PACK", "Noodle Pack", REST),
              ("BERSHKA", "Bershka", CLOTHING),
              ("C & A Moda", "C&A", CLOTHING),
              ("SPIELMANN", "Alimentara Osorhei", GRC),
              ("MADERA MOB", "Tiger Amira", CONSTRUCTION),
              ("Decathlon ", "Decathlon", SPORTS),
              ("COMPEXIT TRADING SRL", "Compexit", "Savings Goal: Car"),
              ("PIATA 9", "Piata 9", REST),
              ("CASIERIE COMPANIA D ORADEA", "Compania de Apa", TAXES),
              ("BILETEONLINE.CFRCALATO", "CFR", TRNSP),
              ("AIRBNB", "AirBNB", HOTELS),
              ("MAROZA", "Magazin Naturist Era", COSMETICS),
              ("RER ORADEA", "RER", TAXES),
              ("elefant.ro", "Elefant.ro", GAMES),
              ("Soldec", "Soldec", HSHLD),
              ("PULL&BEAR", "Pull&Bear", CLOTHING),
              ("KENVELO ", "Kenvelo", CLOTHING),
              ("PROFI MAG", "Profi Vad", GRC),
              ("traistaurbana.ro", "Traista Urbana", CLOTHING),
              ("DEICHMANN ", "Deichmann", CLOTHING),
              ("PEPCO", "Pepco", HSHLD),
              ("LC WAIKIKI", "LC Waikiki", CLOTHING),
              ("TECHNOVA ", "Technova", CONSTRUCTION),
              ("Orsay", "Orsay", CLOTHING),
              
             ]
new_expenses = []
for expense in expenses:
    date, payee, outflow = expense["booking"], expense["partnerName"], expense["amount"]["value"]*1.0/100
    reference = expense["reference"]
    
    for expr in [regex1, regex_foreign, regex_income, regex_online]:
        reg = re.match(expr, reference)
        if reg:
            break
    if reg is None:
        if "Retragere numerar" in reference:
            payee = "Retragere Numerar"
        elif "Incasare Comision de administrare pachet de produse si servicii pt contul" in reference:
            payee = "Comision Card"
        elif "Comision conversie pentru tranzactia cu voucherul" in reference:
            payee = "Comision Card"
        elif "Incasari interbancare in lei" in reference:
            payee = "Comision Card"
        elif "Plus in Cont: Bani inapoi" in reference:
            payee = "BCR"
        elif "Comision de procesare prin sistem de compensare a platilor interbancare de mica valoare pentru" in reference:
            payee = "Comision Card"
        elif "Incasare Comision pentru depasirea pragului de 25 mesaje SMS" in reference:
            payee = "BCR"
        else:
            print("Error: ", payee, reference)
    else:
        payee = reg.group(1)
    category = ''
    memo = []
    # outflow = outflow.replace(".", "").replace(",",".").replace(" RON", "")
    for substr, name, cat in processors:
        if substr in payee:
            payee = name
            category = cat
            break
    else:
        print(payee)
    if float(outflow) > 150:
        memo.append("expensive")

    new_expenses.append([date[:10], payee, category, ",".join(memo), -float(outflow), 0])
    

for expense in new_expenses:
    print(expense)
with open('cleaned.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["Date", "Payee", "Category", "Memo", "Outflow", "Inflow"])
    writer.writerows(new_expenses)

YOUNG BRANDS UOTLET ORADEA.
Error:  UNIVERSITATEA ORADEA plata burse 2019 04
UNIVERSITATEA ORADEA
GB GOOGLE *Google Storage g.co/helpp.
SC RODIA SRL FARMACIA RODORADEA.
TRANSILVANIA NORD MOBI LP MUNICIPIUL.
Error:  UNIVERSITATEA ORADEA plata burse 2019 05
UNIVERSITATEA ORADEA
SC RODIA SRL FARMACIA RODORADEA.
H M RO0067 ORADEA C3 ORADEA.
GB GOOGLE *Google Storage g.co/helpp.
Error:  UNIVERSITATEA ORADEA plata burse 2019 06
UNIVERSITATEA ORADEA
PANEMAR MAG 81 CASA 2 ORADEA.
GB GOOGLE *Google Music g.co/helpp.
PlatiOn*MG NET DIST BUCURESTI.
GB GOOGLE *Google Storage g.co/helpp.
Error:  UNIVERSITATEA ORADEA plata burse 2019 07
UNIVERSITATEA ORADEA
INDIGO COPY CENTER SRL MUNICIPIUL.
KITCHEN SHOP ORADEA DE POradea.
RADA EXIM SRL - RON BH.
['2019-04-20', 'YOUNG BRANDS UOTLET ORADEA.', '', '', 340.0, 0]
['2019-04-20', 'KFC', 'Everyday Expenses: Restaurants', '', 36.0, 0]
['2019-04-20', 'Magazin Naturist Era', 'Everyday Expenses: Cosmetics', '', 246.92, 0]
['2019-04-23', 'UNIVERSITATEA ORADEA',

In [20]:

f = open("./viseca7.csv")

expense_reader = csv.reader(f)
expenses = list(expense_reader)

processors = [("NETFLIX", "Netflix", SUBS), 
              ("Google Commerc", "Google", SUBS),
              ("GOOGLE *Commerc", "Google", SUBS),
              ("GOOGLE *Google Music", "Google", SUBS),
              ("GOOGLE *CDG,", "Google", SUBS),
              ("Coop-3305, Langnau am Al", "Coop Langnau", GRC),
              ("DIGITALOCEAN.COM", "DigitalOcean", SUBS),
              ("Coop", "Coop", GRC),
              ("Migros", "Migros", GRC),
              ("BOUNTYSOURCE", "Bountysource", SUBS),
              ("Amazon", "Amazon", FUN),
              ("AMAZON", "Amazon", FUN),
              ("GOOGLE *Voice", "Google", "Monthly Bills: Internet"), 
              ("GOOGLE *Google Storage", "Google", "Monthly Bills: Internet"), 
              ("UBER", "Uber", TRNSP),
              ("Uber", "Uber", TRNSP),
              ("WP-IFES TRUST", "Debora Roehl", "Giving: Charitable"),
              ("LUFTHANSA", "Lufthansa", TRNSP),
              ("IKEA", "IKEA", HSHLD),
              ("Gidor Coiffure", "Gidor Sihlcity", "Everyday Expenses: Clothing"),
              ("UDACITY, INC.", "Udacity", "Monthly Bills: Subscriptions"),
              ("BEEMINDER", "Beeminder", "Monthly Bills: Subscriptions"),
              ("ITUNES.COM/BILL", "iTunes", FUN),
              ("AIRBNB", "Airbnb", "Fun Stuff: Trips"),
              ("Manor AG - 229, Zuerich", "Manor", REST),
              ("ZUERICH FLUGHAFEN SBB, ZUERICH FLUG", "SBB", TRNSP), 
              ("WWW.ORANGE.RO, BUCURESTI", "Orange", SUBS),
              ("Hiltl Sihlpost, Zurich", "Hiltl", REST),
              ("Swiss smile Schweiz AG", "Swiss Smile", MED),
              ("SCHWEIZ. BUNDESBAHN, BERN", "SBB", TRNSP),
              ("VBZ SCHMIEDE WIEDIKON, ZUERICH", "SBB", TRNSP),
              ("SBB CFF FSS Ticket Shop", "SBB", TRNSP),
              ("ZVV-TICKET-APP, ZUERICH", "SBB", TRNSP), 
              ("PAYPAL *CASAJURNALI", "Casa Jurnalistului", SUBS),
              ("CNP*WIRED MAGAZINE,", "Wired", SUBS)
             ]
new_expenses = []
for expense in expenses:
    date, _, payee, outflow = expense
    category = ''
    memo = []
    outflow = outflow.replace(",","")
    if "Currency rate" in payee:
        for place in ["NEW YORK", "BROOKLYN", "QUEENS", "MOUNTAIN VIEW", "CLUJ NAPOCA", "JAMAICA", "ORADEA", "LONDON"]:
            if place in payee:
                memo.append("Abroad")
                memo.append(place)
    for substr, name, cat in processors:
        if substr in payee:
            payee = name
            category = cat
            break
    else:
        print(payee)
    if float(outflow) > 100:
        memo.append("expensive")

    new_expenses.append([date, payee, category, ",".join(memo), outflow, 0])
    

for expense in new_expenses:
    print(expense)
with open('cleaned.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["Date", "Payee", "Category", "Memo", "Outflow", "Inflow"])
    writer.writerows(new_expenses)


FINETUXEDOS, 9148616000 USD 209.75 Exchange rate 0.9866 (24.06.2017): CHF 206.95 Processing fee 1.75% CHF 3.60
REVOLUT*7419*, REVOLUT.COM 10.10 Processing fee 1.75% CHF 0.20
WIZZ AIR HUN000H9ENSZ, BUDAPEST 136.99 Processing fee 1.75% CHF 2.40
WWW.STARTICKET.CH, ZUERICH
MAD HAIRSTYLING, Zurich
Esprit Store 763 Zurich, Zurich
HOTELS.COM139629236108, HOTELS.COM USD 463.98 Exchange rate 0.9778 (08.07.2017): CHF 453.70 Processing fee 1.75% CHF 7.95
M?V-START, BUDAPEST HUF 4'962.00 Exchange rate 0.0036 (08.07.2017): CHF 18.00 Processing fee 1.75% CHF 0.30
PEEK & CLOPPENBURG, ZUERICH
FO"TAXI, BUDAPEST HUF 4'020.00 Exchange rate 0.0036 (09.07.2017): CHF 14.60 Processing fee 1.75% CHF 0.25
Princess Bakery&Bis, Budapest HUF 455.00 Exchange rate 0.0036 (09.07.2017): CHF 1.65 Processing fee 1.75% CHF 0.05
CROPP TOWN, DEBRECEN HUF 2'995.00 Exchange rate 0.0036 (09.07.2017): CHF 10.85 Processing fee 1.75% CHF 0.20
KFC DEBRECEN, DEBRECEN HUF 3'470.00 Exchange rate 0.0036 (09.07.2017): CHF 12.60 Proce

In [ ]:
f = open("./bcge8.csv")

expense_reader = csv.reader(f)
expenses = list(expense_reader)

processors = [("Muller-Schmidig", "Landlord", "Monthly Bills: Rent"), 
              ("Google","Google", "Income for this month"),
              ("Galaxus", "Digitec", "Fun Stuff: Electronics"),
              ("UPC", "UPC", "Monthly Bills: Internet"),
              ("Elektrizitatswerkedes, Kantons Zurich", "EKZ", "Monthly Bills: Electricity"),
              ("Mobility Genossenschaft, GÃ¼tschstrasse 2, 6 ..", "Mobility", "Everyday Expenses: Transportation"),
              ("TERRE DES HOMMES SCHWEIZ", "Terre Des Hommes", "Giving: Charitable"),
              ("SWICA", "Swica", "Everyday Expenses: Medical"),
              ("ZKB", "Transfer: Cash", ""),
              ("VISECA", "Transfer: Credit Card", ""),
              ("Revolut LTD", "Transfer: Revolut", "")
             ]
new_expenses = []
existing = set()
for expense in expenses:
    date, payee, outflow, inflow, memo = expense
    try:
        date = datetime.datetime.strptime(date, "%d.%m.%Y")
    except:
        date = datetime.datetime.strptime(date, "%d.%m.%y")
    date = date.strftime("%m.%d.%Y")
    outflow = outflow.replace("'","")
    inflow = inflow.replace("'","")
    category = ''
    memo = [] if not memo else [memo]
    for substr, name, cat in processors:
        if substr in payee:
            payee = name
            category = cat
            break
    else:
        print(payee)
    value = inflow if inflow else outflow
    if payee in ["Payment order", "Standing order"] and (date, abs(float(value))) in existing:
        continue                 
        
    existing.add((date, abs(float(value))))
    new_expenses.append([date, payee, category, ",".join(memo), outflow, inflow])

new_expenses.sort(key=lambda x: x[0])
    

for expense in new_expenses:
    print(expense)
with open('cleaned.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["Date", "Payee", "Category", "Memo", "Outflow", "Inflow"])
    writer.writerows(new_expenses)